In [196]:
import pandas as pd
import numpy as np

datapath="../data/"

submission_data = pd.read_csv(datapath + "submission.csv")
test_data = pd.read_csv(datapath + "test.csv")
train_data = pd.read_csv(datapath + "train.csv")
countrycodes=pd.read_csv(datapath+"country-codes_csv.csv")
indicator_data=pd.read_excel(datapath+"Indicator_list.xlsx").fillna("")

indicator_data["long_form"]=indicator_data["long_form"].apply(lambda x: "_".join(x.split(" ")))
train_data[["ConfirmedCases","Fatalities"]]=train_data[["ConfirmedCases","Fatalities"]].astype(int)
submission_data[["ConfirmedCases","Fatalities"]]=submission_data[["ConfirmedCases","Fatalities"]].astype(int)

In [22]:
train_data

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0,0
1,2,NaN,Afghanistan,2020-01-23,0,0
2,3,NaN,Afghanistan,2020-01-24,0,0
3,4,NaN,Afghanistan,2020-01-25,0,0
4,5,NaN,Afghanistan,2020-01-26,0,0
...,...,...,...,...,...,...
19105,29361,NaN,Zimbabwe,2020-03-22,3,0
19106,29362,NaN,Zimbabwe,2020-03-23,3,1
19107,29363,NaN,Zimbabwe,2020-03-24,3,1
19108,29364,NaN,Zimbabwe,2020-03-25,3,1


In [247]:
import requests
from tqdm import tqdm

def import_data(indicator="",indicator_type="",name=""):
    url="https://ghoapi.azureedge.net/api/{}".format(indicator)
    data = pd.DataFrame(requests.get(url).json()["value"])
    data.drop([col for col in data.columns if data[col].isna().sum() == data.shape[0]],axis=1,inplace=True)
    data=data.drop(data[data.NumericValue.isna()].index,axis=0).reset_index(drop=True)
    data=data[data.Dim1==indicator_type] if len(indicator_type)>0 else data
    data=data[["IndicatorCode","SpatialDim","TimeDim","NumericValue"]].drop_duplicates(subset=["IndicatorCode","SpatialDim"],keep="last").reset_index(drop=True)
    data.columns=["IndicatorCode","Shortcut","Year",name]
    
    return data

def import_who_indicators(indicator_data):
    data=[import_data(indicator["short_form"],indicator["addition"],indicator["long_form"]) for _,indicator in tqdm(indicator_data.iterrows())]
    return data
    
who_data_list=import_who_indicators(indicator_data)

11it [00:11,  1.06s/it]


In [250]:
def match_country_code(countrycodes,traindata):
    countrycodes=countrycodes[["ISO3166-1-Alpha-3","official_name_en"]]
    countrycodes=countrycodes.drop(countrycodes[countrycodes["ISO3166-1-Alpha-3"].isna()].index,axis=0).reset_index(drop=True)
    countrycodes.columns = ["Shortcut", "Country_Region"]
    result = traindata.merge(countrycodes,on="Country_Region",how="left").reset_index(drop=True)
    return result

new_traindata = match_country_code(countrycodes,train_data)

def match_dataframes(new_traindata,who_data):
    result = new_traindata.merge(who_data,on="Shortcut",how="left")
    return result

test_traindata = new_traindata.copy()
for who_data in who_data_list:
    test_traindata=match_dataframes(test_traindata,who_data)

In [253]:
test_traindata

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,Shortcut,IndicatorCode_x,Year_x,Population_using_at_least_basic_sanitation_services_(%),...,Raised_fasting_blood_glucose_(>=7.0_mmol/L_or_on_medication)(age-standardized_estimate),IndicatorCode_x,Year_x,Raised_blood_pressure_(SBP>=140_OR_DBP>=90)_(age-standardized_estimate),IndicatorCode_y,Year_y,"Prevalence_of_obesity_among_adults,_BMI_>=_30_(age-standardized_estimate)_(%)",IndicatorCode,Year,Health_Service_Provision
0,1,NaN,Afghanistan,2020-01-22,0,0,AFG,WSH_SANITATION_BASIC,2017.0,62.26144,...,11.9,BP_04,2015.0,30.6,NCD_BMI_30A,2016.0,3.2,IHRSPAR_CAPACITY09,2018.0,60.0
1,2,NaN,Afghanistan,2020-01-23,0,0,AFG,WSH_SANITATION_BASIC,2017.0,62.26144,...,11.9,BP_04,2015.0,30.6,NCD_BMI_30A,2016.0,3.2,IHRSPAR_CAPACITY09,2018.0,60.0
2,3,NaN,Afghanistan,2020-01-24,0,0,AFG,WSH_SANITATION_BASIC,2017.0,62.26144,...,11.9,BP_04,2015.0,30.6,NCD_BMI_30A,2016.0,3.2,IHRSPAR_CAPACITY09,2018.0,60.0
3,4,NaN,Afghanistan,2020-01-25,0,0,AFG,WSH_SANITATION_BASIC,2017.0,62.26144,...,11.9,BP_04,2015.0,30.6,NCD_BMI_30A,2016.0,3.2,IHRSPAR_CAPACITY09,2018.0,60.0
4,5,NaN,Afghanistan,2020-01-26,0,0,AFG,WSH_SANITATION_BASIC,2017.0,62.26144,...,11.9,BP_04,2015.0,30.6,NCD_BMI_30A,2016.0,3.2,IHRSPAR_CAPACITY09,2018.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19105,29361,NaN,Zimbabwe,2020-03-22,3,0,ZWE,WSH_SANITATION_BASIC,2017.0,46.21599,...,7.1,BP_04,2015.0,28.2,NCD_BMI_30A,2016.0,4.7,IHRSPAR_CAPACITY09,2018.0,20.0
19106,29362,NaN,Zimbabwe,2020-03-23,3,1,ZWE,WSH_SANITATION_BASIC,2017.0,46.21599,...,7.1,BP_04,2015.0,28.2,NCD_BMI_30A,2016.0,4.7,IHRSPAR_CAPACITY09,2018.0,20.0
19107,29363,NaN,Zimbabwe,2020-03-24,3,1,ZWE,WSH_SANITATION_BASIC,2017.0,46.21599,...,7.1,BP_04,2015.0,28.2,NCD_BMI_30A,2016.0,4.7,IHRSPAR_CAPACITY09,2018.0,20.0
19108,29364,NaN,Zimbabwe,2020-03-25,3,1,ZWE,WSH_SANITATION_BASIC,2017.0,46.21599,...,7.1,BP_04,2015.0,28.2,NCD_BMI_30A,2016.0,4.7,IHRSPAR_CAPACITY09,2018.0,20.0


In [249]:
list(set(who_data_list[6].Year))

[2025, 2012]